In [1]:
import os
import numpy as np
import networkx as nx
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from collections import Counter

# Imposta la variabile per limitare il numero di core utilizzati
os.environ["LOKY_MAX_CPU_COUNT"] = "4"  # Sostituisci "4" con il numero di core che desideri utilizzare

import sys
project_path = os.getcwd().split("Code")[0]
sys.path.append(project_path)

# Ora puoi importare la classe GraphConstructor
from Code.notebook.graph.GraphConstructor import GraphConstructor

# Funzione per eseguire il clustering basato su modelli (Gaussian Mixture Model)
def model_based_clustering(graph, num_clusters):
    adjacency_matrix = nx.to_numpy_array(graph)
    distance_matrix = np.max(adjacency_matrix) - adjacency_matrix  # Inverti la matrice di adiacenza per ottenere una matrice di distanze
    gmm = GaussianMixture(n_components=num_clusters, covariance_type='full', random_state=42, n_init=1)
    gmm.fit(distance_matrix)
    cluster_labels = gmm.predict(distance_matrix)
    return {node: label for node, label in zip(graph.nodes, cluster_labels)}

# Funzione per sanitizzare gli attributi del grafo (nodi e archi)
def sanitize_graph_attributes(graph):
    for node in graph.nodes:
        attrs = graph.nodes[node]
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.nodes[node].clear()
        graph.nodes[node].update(sanitized_attrs)

    for u, v, attrs in graph.edges(data=True):
        sanitized_attrs = {}
        for key, value in attrs.items():
            new_key = str(key).replace(" ", "_").replace("-", "_")
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.edges[u, v].clear()
        graph.edges[u, v].update(sanitized_attrs)

# Funzione per determinare il numero ottimale di cluster usando il metodo dell'Elbow
def optimal_num_clusters(graph, max_clusters=10):
    adjacency_matrix = nx.to_numpy_array(graph)
    distance_matrix = np.max(adjacency_matrix) - adjacency_matrix
    inertias = []
    for k in range(1, max_clusters + 1):
        gmm = GaussianMixture(n_components=k, covariance_type='full', random_state=42, n_init=1)
        gmm.fit(distance_matrix)
        inertias.append(gmm.score(distance_matrix))
    optimal_clusters = inertias.index(max(inertias)) + 1
    return optimal_clusters

# Crea il grafo utilizzando GraphConstructor
graph_builder = GraphConstructor()
graph_builder.build_graph()
graph = graph_builder.graph

# Calcola il numero ottimale di cluster usando il metodo dell'Elbow
optimal_clusters = optimal_num_clusters(graph)

# Esegui il clustering basato su modello (GMM)
gmm_labels = model_based_clustering(graph, optimal_clusters)

# Copia il grafo originale per assegnare i cluster
gr = graph.copy()

# Assegna il cluster a ciascun nodo
for node, label in gmm_labels.items():
    gr.nodes[node]['cluster'] = label

# Sanitizza gli attributi del grafo prima di esportarlo
sanitize_graph_attributes(gr)

# Esporta il grafo in formato GML
output_path = "graph_with_gmm_clusters.gml"
nx.write_gml(gr, output_path)
